# 04: All Contract Types — The Complete Reference

KindScript enforces 5 types of architectural contracts. Each one catches a different class of violation:

| Contract | Error Code | What it catches |
|----------|------------|----------------|
| `noDependency` | KS70001 | Forbidden imports between layers |
| `mustImplement` | KS70002 | Missing interface implementations |
| `purity` | KS70003 | I/O imports in pure layers |
| `noCycles` | KS70004 | Circular dependencies between layers |
| `colocated` | KS70005 | Missing counterpart files (e.g., tests) |

This notebook demonstrates each one with a **violation** and a **fix**.

## Setup

In [ ]:
const PROJECT_ROOT = Deno.cwd().replace(/\/notebooks$/, "");
const KSC = PROJECT_ROOT + "/dist/infrastructure/cli/main.js";

async function ksc(...args: string[]): Promise<{ code: number; output: string }> {
  const cmd = new Deno.Command("node", {
    args: [KSC, ...args],
    stdout: "piped",
    stderr: "piped",
  });
  const { code, stdout, stderr } = await cmd.output();
  const output = (new TextDecoder().decode(stdout) + new TextDecoder().decode(stderr)).trim();
  if (output) console.log(output);
  console.log(`\nExit code: ${code}`);
  return { code, output };
}

// Boilerplate that goes at the top of every architecture.ts
const BOILERPLATE = `
interface Kind<N extends string = string> {
  readonly kind: N;
  readonly location: string;
}

interface ContractConfig {
  noDependency?: [string, string][];
  mustImplement?: [string, string][];
  purity?: string[];
  noCycles?: string[];
  colocated?: [string, string][];
}
function defineContracts<_T = unknown>(config: ContractConfig): ContractConfig {
  return config;
}
`.trimStart();

const KINDSCRIPT_JSON = JSON.stringify({ definitions: ["architecture.ts"] }, null, 2);
const TSCONFIG = JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: "src", outDir: "dist" },
  include: ["src/**/*.ts"],
}, null, 2);

function makeProject(): string {
  const dir = Deno.makeTempDirSync({ prefix: "ksc-contracts-" });
  Deno.writeTextFileSync(`${dir}/kindscript.json`, KINDSCRIPT_JSON);
  Deno.writeTextFileSync(`${dir}/tsconfig.json`, TSCONFIG);
  return dir;
}

console.log("Setup complete.");

---

## A. `noDependency` — Forbidden Imports (KS70001)

The most common contract. Forbids imports from one layer to another.

**Use case:** In Clean Architecture, the domain layer must not import from infrastructure. The business logic should never know about databases, HTTP clients, or file systems.

```typescript
noDependency: [
  ["domain", "infrastructure"],  // domain cannot import from infrastructure
]
```

In [ ]:
// Create a project where domain imports from infrastructure
const demo1 = makeProject();
Deno.mkdirSync(`${demo1}/src/domain`, { recursive: true });
Deno.mkdirSync(`${demo1}/src/infrastructure`, { recursive: true });

// The violation: domain service imports from infrastructure
Deno.writeTextFileSync(`${demo1}/src/domain/service.ts`, `
import { Database } from '../infrastructure/database';

export class DomainService {
  private db = new Database();
  getAll(): string[] {
    return this.db.query('SELECT * FROM entities');
  }
}
`.trimStart());

Deno.writeTextFileSync(`${demo1}/src/infrastructure/database.ts`, `
export class Database {
  query(sql: string): string[] {
    return [sql];
  }
}
`.trimStart());

Deno.writeTextFileSync(`${demo1}/architecture.ts`, BOILERPLATE + `
export interface CleanContext extends Kind<"CleanContext"> {
  domain: DomainLayer;
  infrastructure: InfrastructureLayer;
}

export interface DomainLayer extends Kind<"DomainLayer"> {}
export interface InfrastructureLayer extends Kind<"InfrastructureLayer"> {}

export const app: CleanContext = {
  kind: "CleanContext",
  location: "src",
  domain: { kind: "DomainLayer", location: "src/domain" },
  infrastructure: { kind: "InfrastructureLayer", location: "src/infrastructure" },
};

export const contracts = defineContracts<CleanContext>({
  noDependency: [["domain", "infrastructure"]],
});
`);

console.log("=== Violation: domain imports from infrastructure ===");
await ksc("check", demo1);

In [ ]:
// Fix: domain uses its own interface, infrastructure provides the implementation
Deno.writeTextFileSync(`${demo1}/src/domain/service.ts`, `
// Domain defines its OWN interface — no infrastructure import
export interface DataStore {
  query(sql: string): string[];
}

export class DomainService {
  constructor(private store: DataStore) {}
  getAll(): string[] {
    return this.store.query('SELECT * FROM entities');
  }
}
`.trimStart());

console.log("=== Fixed: domain uses its own interface ===");
await ksc("check", demo1);

Deno.removeSync(demo1, { recursive: true });

---

## B. `purity` — No I/O in Pure Layers (KS70003)

Ensures a layer has no side effects — no `fs`, `http`, `net`, `child_process`, or any of Node's ~50 built-in I/O modules.

**Use case:** The domain layer should be pure functions and data. If it needs to read a file, it should receive the data through a port — not call `fs.readFileSync` directly.

```typescript
purity: ["domain"]  // domain can't import fs, http, etc.
```

In [ ]:
// Create a project where domain imports Node.js fs module
const demo2 = makeProject();
Deno.mkdirSync(`${demo2}/src/domain`, { recursive: true });

// The violation: domain service reads files directly
Deno.writeTextFileSync(`${demo2}/src/domain/service.ts`, `
import * as fs from 'fs';

export class DomainService {
  readData(): string {
    return fs.readFileSync('/tmp/data.txt', 'utf-8');
  }
}
`.trimStart());

Deno.writeTextFileSync(`${demo2}/architecture.ts`, BOILERPLATE + `
export interface AppContext extends Kind<"AppContext"> {
  domain: DomainLayer;
}

export interface DomainLayer extends Kind<"DomainLayer"> {}

export const app: AppContext = {
  kind: "AppContext",
  location: "src",
  domain: { kind: "DomainLayer", location: "src/domain" },
};

export const contracts = defineContracts<AppContext>({
  purity: ["domain"],
});
`);

console.log("=== Violation: domain imports 'fs' ===");
await ksc("check", demo2);

In [ ]:
// Fix: inject the data instead of reading it directly
Deno.writeTextFileSync(`${demo2}/src/domain/service.ts`, `
// Pure domain: receives data through constructor, no I/O
export interface DataReader {
  read(path: string): string;
}

export class DomainService {
  constructor(private reader: DataReader) {}
  readData(): string {
    return this.reader.read('/tmp/data.txt');
  }
}
`.trimStart());

console.log("=== Fixed: domain uses injected reader, no direct I/O ===");
await ksc("check", demo2);

Deno.removeSync(demo2, { recursive: true });

---

## C. `mustImplement` — Every Port Needs an Adapter (KS70002)

Ensures every exported interface in one layer has a class that `implements` it in another layer.

**Use case:** In Hexagonal Architecture, ports are interfaces defined in the domain. Adapters are concrete implementations. This contract guarantees you haven't defined a port without wiring up an adapter.

```typescript
mustImplement: [["ports", "adapters"]]  // every interface in ports needs a class in adapters
```

In [ ]:
// Create a project with a port but no adapter
const demo3 = makeProject();
Deno.mkdirSync(`${demo3}/src/ports`, { recursive: true });
Deno.mkdirSync(`${demo3}/src/adapters`, { recursive: true });

// Port interface
Deno.writeTextFileSync(`${demo3}/src/ports/repository.port.ts`, `
export interface RepositoryPort {
  save(entity: unknown): void;
  findAll(): unknown[];
}
`.trimStart());

// Adapters directory exists but has no implementing class
Deno.writeTextFileSync(`${demo3}/src/adapters/placeholder.ts`, `
export const placeholder = true;
`.trimStart());

Deno.writeTextFileSync(`${demo3}/architecture.ts`, BOILERPLATE + `
export interface AppContext extends Kind<"AppContext"> {
  ports: PortsLayer;
  adapters: AdaptersLayer;
}

export interface PortsLayer extends Kind<"PortsLayer"> {}
export interface AdaptersLayer extends Kind<"AdaptersLayer"> {}

export const app: AppContext = {
  kind: "AppContext",
  location: "src",
  ports: { kind: "PortsLayer", location: "src/ports" },
  adapters: { kind: "AdaptersLayer", location: "src/adapters" },
};

export const contracts = defineContracts<AppContext>({
  mustImplement: [["ports", "adapters"]],
});
`);

console.log("=== Violation: RepositoryPort has no adapter ===");
await ksc("check", demo3);

In [ ]:
// Fix: add an adapter that implements the port
Deno.writeTextFileSync(`${demo3}/src/adapters/repository.adapter.ts`, `
import { RepositoryPort } from '../ports/repository.port';

export class InMemoryRepositoryAdapter implements RepositoryPort {
  private store: unknown[] = [];

  save(entity: unknown): void {
    this.store.push(entity);
  }

  findAll(): unknown[] {
    return [...this.store];
  }
}
`.trimStart());

console.log("=== Fixed: InMemoryRepositoryAdapter implements RepositoryPort ===");
await ksc("check", demo3);

Deno.removeSync(demo3, { recursive: true });

---

## D. `noCycles` — No Circular Dependencies (KS70004)

Detects circular dependency chains between layers. If domain imports from infrastructure AND infrastructure imports from domain, you have a cycle that makes both impossible to test in isolation.

```typescript
noCycles: ["domain", "infrastructure"]  // check for cycles between these layers
```

In [ ]:
// Create a project with a circular dependency
const demo4 = makeProject();
Deno.mkdirSync(`${demo4}/src/domain`, { recursive: true });
Deno.mkdirSync(`${demo4}/src/infrastructure`, { recursive: true });

// Domain imports from infrastructure
Deno.writeTextFileSync(`${demo4}/src/domain/service.ts`, `
import { Database } from '../infrastructure/database';

export class DomainService {
  private db = new Database();
  getData(): string[] {
    return this.db.query('SELECT * FROM data');
  }
}
`.trimStart());

// Infrastructure imports from domain — completing the cycle!
Deno.writeTextFileSync(`${demo4}/src/infrastructure/database.ts`, `
import { DomainService } from '../domain/service';

export class Database {
  private service = new DomainService();
  query(sql: string): string[] {
    return [sql];
  }
}
`.trimStart());

Deno.writeTextFileSync(`${demo4}/architecture.ts`, BOILERPLATE + `
export interface AppContext extends Kind<"AppContext"> {
  domain: DomainLayer;
  infrastructure: InfrastructureLayer;
}

export interface DomainLayer extends Kind<"DomainLayer"> {}
export interface InfrastructureLayer extends Kind<"InfrastructureLayer"> {}

export const app: AppContext = {
  kind: "AppContext",
  location: "src",
  domain: { kind: "DomainLayer", location: "src/domain" },
  infrastructure: { kind: "InfrastructureLayer", location: "src/infrastructure" },
};

export const contracts = defineContracts<AppContext>({
  noCycles: ["domain", "infrastructure"],
});
`);

console.log("=== Violation: domain <-> infrastructure cycle ===");
await ksc("check", demo4);

In [ ]:
// Fix: break the cycle — infrastructure can depend on domain, but not the reverse
Deno.writeTextFileSync(`${demo4}/src/domain/service.ts`, `
// Domain defines its own interface — no infrastructure import
export interface DataStore {
  query(sql: string): string[];
}

export class DomainService {
  constructor(private store: DataStore) {}
  getData(): string[] {
    return this.store.query('SELECT * FROM data');
  }
}
`.trimStart());

Deno.writeTextFileSync(`${demo4}/src/infrastructure/database.ts`, `
import { DataStore } from '../domain/service';

export class Database implements DataStore {
  query(sql: string): string[] {
    return [sql];
  }
}
`.trimStart());

console.log("=== Fixed: one-directional dependency (infra -> domain) ===");
await ksc("check", demo4);

Deno.removeSync(demo4, { recursive: true });

---

## E. `colocated` — Every File Needs a Counterpart (KS70005)

Ensures files in one directory have matching filenames in another directory. This is the "every component has a test" contract.

```typescript
colocated: [["components", "tests"]]  // every component needs a test file
```

In [ ]:
// Create a project where one component is missing its test
const demo5 = makeProject();
Deno.mkdirSync(`${demo5}/src/components`, { recursive: true });
Deno.mkdirSync(`${demo5}/src/tests`, { recursive: true });

// Two components
Deno.writeTextFileSync(`${demo5}/src/components/button.ts`, `
export function Button() { return 'button'; }
`.trimStart());

Deno.writeTextFileSync(`${demo5}/src/components/form.ts`, `
export function Form() { return 'form'; }
`.trimStart());

// Only one test — button has a test, form does not
Deno.writeTextFileSync(`${demo5}/src/tests/button.ts`, `
import { Button } from '../components/button';
console.assert(Button() === 'button');
`.trimStart());

Deno.writeTextFileSync(`${demo5}/architecture.ts`, BOILERPLATE + `
export interface AppContext extends Kind<"AppContext"> {
  components: ComponentsLayer;
  tests: TestsLayer;
}

export interface ComponentsLayer extends Kind<"ComponentsLayer"> {}
export interface TestsLayer extends Kind<"TestsLayer"> {}

export const app: AppContext = {
  kind: "AppContext",
  location: "src",
  components: { kind: "ComponentsLayer", location: "src/components" },
  tests: { kind: "TestsLayer", location: "src/tests" },
};

export const contracts = defineContracts<AppContext>({
  colocated: [["components", "tests"]],
});
`);

console.log("=== Violation: form.ts has no counterpart test ===");
await ksc("check", demo5);

In [ ]:
// Fix: add the missing test
Deno.writeTextFileSync(`${demo5}/src/tests/form.ts`, `
import { Form } from '../components/form';
console.assert(Form() === 'form');
`.trimStart());

console.log("=== Fixed: form.ts now has a test ===");
await ksc("check", demo5);

Deno.removeSync(demo5, { recursive: true });

---

## Summary

| Contract | Code | Catches | Fix pattern |
|----------|------|---------|-------------|
| `noDependency` | KS70001 | Forbidden imports between layers | Use dependency injection — define interfaces in the inner layer |
| `mustImplement` | KS70002 | Port interfaces without adapters | Add a class that `implements` the interface |
| `purity` | KS70003 | I/O imports (`fs`, `http`, etc.) in pure layers | Inject I/O through constructor instead of importing directly |
| `noCycles` | KS70004 | Circular dependency chains | Break the cycle with interfaces (Dependency Inversion) |
| `colocated` | KS70005 | Missing counterpart files | Add the missing file (test, handler, etc.) |

Contracts can be combined. A real project typically uses `noDependency` + `purity` at minimum, and adds `mustImplement` for hexagonal architectures and `colocated` for test coverage enforcement.

```typescript
export const contracts = defineContracts<CleanContext>({
  noDependency: [
    ["domain", "infrastructure"],
    ["domain", "application"],
    ["application", "infrastructure"],
  ],
  purity: ["domain"],
  mustImplement: [["ports", "adapters"]],
  colocated: [["components", "tests"]],
});
```